### ECON 323 FINAL PROJECT

In [2]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [3]:
# reading in csv data 

results15 = pd.read_csv("DATA/2015Results.csv")
results19 = pd.read_csv("DATA/2019Results.csv")
results19.head()

NameError: name 'pd' is not defined

In [4]:
# renaming columns so that dataframes can be concatenated correctly 
Results15 = results15.rename(columns = {"RidingID":"Riding ID", "ranfor43":"RanForNE", "reelected43":"Reelected", "DOB1":"DOB", "YearsServedAtTimeOfElection":"YearsInHOC"})
Results19 = results19.rename(columns = {"Election":"Parliament", "ranfor44":"RanForNE", "reelected44":"Reelected", "StatementsByMembers":"Statements", "YearsServedAtElection":"YearsInHOC"})

In [29]:
# concatenating dataframes
Results_Full = pd.concat([Results15, Results19], ignore_index=True)
Results_Full
# combining 2015 and 2019 datasets leds to MPs showing up twice -- doesn't matter for analysis because each row is just one observation - person itself doesnt matter

,Name,Party,Parliament,Politician ID,Riding ID,Province,Gender,RanForNE,Reelected,QuestionPeriod,PvtMemBills,Statements,Backbencher,DOB,AgeOnElectionDay,YearsInHOC
0,Dean Allison,Conservative Party of Canada,42,5,35068,ON,M,Yes,Yes,42,0,19,0,1965-02-18,54.0,15.323288
1,Rona Ambrose,Conservative Party of Canada,42,6,48033,AB,F,No,.,457,1,3,0,1969-03-15,50.0,15.323288
2,David Anderson,Conservative Party of Canada,42,9,47002,SK,M,No,.,53,1,20,0,1957-08-15,62.0,18.909589
3,Charlie Angus,New Democratic Party,42,11,35107,ON,M,Yes,Yes,168,0,15,1,1962-11-14,56.0,15.323288
4,Larry Bagnell,Liberal Party of Canada,42,15,60001,YT,M,Yes,Yes,11,1,15,1,1949-12-19,69.0,16.350685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Maninder Sidhu,Liberal Party of Canada,43,12210,35009,ON,M,Yes,Yes,5,0,3,1,1984-04-26,37.0,1.917808
692,Nelly Shin,Conservative Party of Canada,43,12222,59023,BC,F,Yes,No,13,0,6,0,1971-01-01,50.0,1.917808
693,Rob Morrison,Conservative Party of Canada,43,12228,59015,BC,M,Yes,Yes,10,0,4,0,1956-05-03,65.0,1.917808
694,Marci Ien,Liberal Party of Canada,43,12231,35108,ON,F,Yes,Yes,4,0,2,1,1969-07-29,52.0,0.901370


In [30]:
Results_Dummy = pd.DataFrame(data=Results_Full, columns=['Gender', 'RanForNE', 'Reelected', 'Party'])
Results_Dummy

just_dummies = pd.get_dummies(Results_Dummy)
just_dummies

Dummies = just_dummies.drop(columns=['Gender_M', 'RanForNE_No', 'Reelected_.', 'Reelected_No', 'Party_Bloc Québécois', 'Party_Green Party of Canada', 'Party_Independent'])
Dummies

Results1 = pd.concat([Results_Full, Dummies], axis = 1)
Results1

,Name,Party,Parliament,Politician ID,Riding ID,Province,Gender,RanForNE,Reelected,QuestionPeriod,...,Backbencher,DOB,AgeOnElectionDay,YearsInHOC,Gender_F,RanForNE_Yes,Reelected_Yes,Party_Conservative Party of Canada,Party_Liberal Party of Canada,Party_New Democratic Party
0,Dean Allison,Conservative Party of Canada,42,5,35068,ON,M,Yes,Yes,42,...,0,1965-02-18,54.0,15.323288,0,1,1,1,0,0
1,Rona Ambrose,Conservative Party of Canada,42,6,48033,AB,F,No,.,457,...,0,1969-03-15,50.0,15.323288,1,0,0,1,0,0
2,David Anderson,Conservative Party of Canada,42,9,47002,SK,M,No,.,53,...,0,1957-08-15,62.0,18.909589,0,0,0,1,0,0
3,Charlie Angus,New Democratic Party,42,11,35107,ON,M,Yes,Yes,168,...,1,1962-11-14,56.0,15.323288,0,1,1,0,0,1
4,Larry Bagnell,Liberal Party of Canada,42,15,60001,YT,M,Yes,Yes,11,...,1,1949-12-19,69.0,16.350685,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Maninder Sidhu,Liberal Party of Canada,43,12210,35009,ON,M,Yes,Yes,5,...,1,1984-04-26,37.0,1.917808,0,1,1,0,1,0
692,Nelly Shin,Conservative Party of Canada,43,12222,59023,BC,F,Yes,No,13,...,0,1971-01-01,50.0,1.917808,1,1,0,1,0,0
693,Rob Morrison,Conservative Party of Canada,43,12228,59015,BC,M,Yes,Yes,10,...,0,1956-05-03,65.0,1.917808,0,1,1,1,0,0
694,Marci Ien,Liberal Party of Canada,43,12231,35108,ON,F,Yes,Yes,4,...,1,1969-07-29,52.0,0.901370,1,1,1,0,1,0


In [35]:
Results = Results1.drop(columns=['Gender', 'RanForNE', 'Reelected', 'Party', 'DOB'])
RESULTS = Results.rename(columns = {"Gender_F":"Gender", "RanForNE_Yes":"RanForNE", "Reelected_Yes":"Reelected", "Party_Conservative Party of Canada":"ConservativeMem", "Party_Liberal Party of Canada":"LiberalMem", "Party_New Democratic Party":"NDPMem"})

RESULTS['QuestionsLog'] = np.log10(RESULTS['QuestionPeriod'])
RESULTS = RESULTS.drop(RESULTS[RESULTS.RanForNE == 0].index)
RESULTS

/Users/anahat/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Name,Parliament,Politician ID,Riding ID,Province,QuestionPeriod,PvtMemBills,Statements,Backbencher,AgeOnElectionDay,YearsInHOC,Gender,RanForNE,Reelected,ConservativeMem,LiberalMem,NDPMem,QuestionsLog
0,Dean Allison,42,5,35068,ON,42,0,19,0,54.0,15.323288,0,1,1,1,0,0,1.623249
3,Charlie Angus,42,11,35107,ON,168,0,15,1,56.0,15.323288,0,1,1,0,0,1,2.225309
4,Larry Bagnell,42,15,60001,YT,11,1,15,1,69.0,16.350685,0,1,1,0,1,0,1.041393
5,Navdeep Bains,42,16,35062,ON,468,0,0,0,42.0,12.767123,0,1,1,0,1,0,2.670246
7,Carolyn Bennett,42,24,35090,ON,217,0,0,0,68.0,22.400000,1,1,1,0,1,0,2.336460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,Maninder Sidhu,43,12210,35009,ON,5,0,3,1,37.0,1.917808,0,1,1,0,1,0,0.698970
692,Nelly Shin,43,12222,59023,BC,13,0,6,0,50.0,1.917808,1,1,0,1,0,0,1.113943
693,Rob Morrison,43,12228,59015,BC,10,0,4,0,65.0,1.917808,0,1,1,1,0,0,1.000000
694,Marci Ien,43,12231,35108,ON,4,0,2,1,52.0,0.901370,1,1,1,0,1,0,0.602060
